In [4]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import cv2
from skimage import measure, segmentation
from vis_utils import load_volume, VolumeVisualizer
from scipy.ndimage import zoom

In [5]:
source_dir = './data/'
files = glob.glob(source_dir + '/*/*.raw')
files[0]

'./data/P12/P12_60um_1333x443x864.raw'

In [6]:
%%time
sample_volume = load_volume(files[0])
print(sample_volume.shape)

(864, 443, 1333)
CPU times: user 0 ns, sys: 239 ms, total: 239 ms
Wall time: 241 ms


In [7]:
# visualizer = VolumeVisualizer(sample_volume).visualize(scale=0.6)

In [8]:
og_binary_mask = (sample_volume > 80).astype(np.uint8)
og_binary_mask.shape

(864, 443, 1333)

In [9]:
# visualizer = VolumeVisualizer(og_vol_binary_mask).visualize(scale=0.6, primary_color=(255,255,255))

In [10]:
def get_main_regions_zoomed_seeds(og_binary_mask, zoom_value=2, size_threshold=5000):
    zoomed_binary_mask = zoom(og_binary_mask, 1 / zoom_value, order=0)
    zoomed_binary_mask_regions = measure.label(zoomed_binary_mask)
    print('number of found regions is', np.max(zoomed_binary_mask_regions))
    zoomed_binary_mask_regionprops = measure.regionprops(zoomed_binary_mask_regions)
    
    main_regions_zoomed_seeds = []
    for reg in zoomed_binary_mask_regionprops:
        if(reg.filled_area > size_threshold):
            main_regions_zoomed_seeds.append(np.argwhere(zoomed_binary_mask == reg.label)[0])
    return main_regions_zoomed_seeds

In [11]:
zoom_value = 2
size_threshold = 10000
main_regions_zoomed_seeds = get_main_regions_zoomed_seeds(og_binary_mask, zoom_value, size_threshold)
main_regions_zoomed_seeds

number of found regions is 385


[array([ 15,  74, 211])]

In [ ]:
def get_main_regions_seeds(main_regions_zoomed_seeds, zoom_value):
    main_regions_seeds = []
    for zoomed_seed in main_regions_zoomed_seeds:
        main_regions_seeds.append(zoomed_seed * zoom_value)
        # TODO mozliwe ze trzeba ogarnac szescian
    return main_regions_seeds

In [20]:
main_regions_seeds = get_main_regions_seeds(main_regions_zoomed_seeds, zoom_value)
main_regions_seeds

[array([ 30, 148, 422])]

In [28]:
def get_main_region_masks(og_binary_mask, main_regions_seeds):
    main_regions = []
    for seed in main_regions_seeds:
        if og_binary_mask[tuple(seed)] == 0:
            raise Exception("seed point set to background")
        region = segmentation.flood(og_binary_mask, tuple(seed))
    return main_regions

In [29]:
xd = get_main_region_masks(og_binary_mask, main_regions_seeds)

Exception: raczychho

In [ ]:
xd


In [12]:
def get_main_region_masks(vol_binary_mask, size_threshold):
    vol_binary_mask = vol_binary_mask.copy()
    seeds = np.argwhere(vol_binary_mask == 1)
    main_regions = []
    for seed in seeds:
        if vol_binary_mask[tuple(seed)] == 0:
            continue
        region = segmentation.flood(vol_binary_mask, tuple(seed))
        vol_binary_mask -= region
        if np.count_nonzero(region) > size_threshold:
            main_regions.append(region)
    return main_regions

In [5]:
def get_main_region_masks(vol_binary_mask, size_threshold):
    vol_binary_mask = vol_binary_mask.copy()
    main_regions = []
    while True:
        print("dick")
        seeds = np.argwhere(vol_binary_mask == 1)
        if seeds.size == 0:
            break
        print(seeds[0])
        region = segmentation.flood(vol_binary_mask, tuple(seeds[0]))
        vol_binary_mask -= region
        if np.count_nonzero(region) > size_threshold:
            main_regions.append(region)
    return main_regions

In [6]:
main_region_masks = get_main_region_masks(og_vol_binary_mask, 10000)
len(main_region_masks)

dick
[ 28 149 425]
dick
[ 37 150 402]
dick
[ 68 214 579]
dick
[ 70 111 370]
dick
[ 72 213 580]
dick
[ 74 182 304]
dick
[ 75 183 306]
dick
[ 76 183 308]
dick
[ 81 207 666]
dick
[ 90  71 350]
dick
[ 93 215 711]
dick
[ 95  74 294]
dick
[113 206 553]
dick
[119 232 806]
dick
[122  67 360]
dick
[122 199 297]
dick
[131 114 278]
dick
[136  53 359]
dick
[144 133 300]
dick
[145 177 712]
dick
[146 101 268]
dick
[146 249 566]
dick
[148 273 628]
dick
[150  63 406]
dick
[150 250 712]
dick
[152 273 486]
dick
[153 224 458]
dick
[153 250 436]
dick
[154 244 364]
dick
[156 235 705]
dick
[157  54 256]
dick
[159 148 289]
dick
[159 215 451]
dick
[160 142 237]
dick
[162 248 721]
dick
[164 293 729]
dick
[168 129 229]
dick
[170 211 352]
dick
[171 309 823]
dick
[176  61 349]
dick
[184 170 260]
dick
[184 284 735]
dick
[ 188  224 1107]
dick
[190 111 277]
dick
[198 143 324]
dick
[201 151 362]
dick
[201 319 918]
dick
[203 296 854]
dick
[205 269 238]
dick
[208 163 352]
dick
[209 164 362]
dick
[ 214  248 1097]
dick
[

KeyboardInterrupt: 

In [22]:
visualizer = VolumeVisualizer(main_region).visualize(primary_color=(255,255,255))

In [16]:
first_reg = (regions == 1).astype(np.uint8) * 255

In [17]:
visualizer = VolumeVisualizer(first_reg)
visualizer.visualize(primary_color=(255,255,255))

In [52]:
tuple(seeds[0])

(22, 119, 340)

In [53]:
zoomed_binary_mask.size

260758524

In [47]:
main_region = segmentation.flood(zoomed_binary_mask, tuple(seeds[0]))

In [51]:
main_region.size

260758524

In [50]:
main_region_regions = measure.label(main_region)
# main_region_regionprops = measure.regionprops(main_region_regions)
main_region_regions.max()

MemoryError: Unable to allocate 1.94 GiB for an array with shape (260758524,) and data type int64